In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     |████████████████████████████████| 2.3MB 123kB/s 


In [ ]:
import os
import sys

from urllib.request import urlretrieve

import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
Biopython version: 1.77


**Пребарување на рекорди за анемија кај луѓе**

In [ ]:
from Bio import Entrez
Entrez.email='naceskikosta@yahoo.com'
handle = Entrez.esearch(db = 'nucleotide',term='sickle AND homo sapiens AND globin NOT chromosome ')
record = Entrez.read(handle)
handle.close()
print(record['Count'],record['IdList'])



6 ['1515564438', '179408', '224959855', '2168937', '183859', '183844']


In [ ]:
from Bio import SeqIO
Entrez.email='naceskikosta@yahoo.com'
with Entrez.efetch(
    db="nucleotide", rettype="fasta", retmode="text", id="224959855"
) as handle:
    seq_record = SeqIO.read(handle, "fasta")
print("%s with %i features" % (seq_record.id, len(seq_record.features)))


FJ766333.1 with 0 features


In [ ]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "A.N.Other@example.com"
with Entrez.efetch(
    db="nucleotide", rettype="gb", retmode="text", id="1515564438,179408,224959855,2168937,183859,183844"
) as handle:
    for seq_record in SeqIO.parse(handle, "gb"):
        print("%s %s..." % (seq_record.id, seq_record.description[:50]))
        print(
            "Sequence length %i, %i features, from: %s"
            % (
                len(seq_record),
                len(seq_record.features),
                seq_record.annotations["source"],
            )
        )

MH580289.1 Homo sapiens voucher ATGLAB 2018103 hemoglobin bet...
Sequence length 881, 5 features, from: Homo sapiens (human)
M25079.1 Human sickle cell beta-globin mRNA, complete cds...
Sequence length 468, 2 features, from: Homo sapiens (human)
FJ766333.1 Homo sapiens A-gamma globin (HBG1) gene, promoter ...
Sequence length 88, 3 features, from: Homo sapiens (human)
E00658.1 Part of DNA encoding beta-globin gene...
Sequence length 50, 1 features, from: Homo sapiens (human)
M33706.1 Human hemoglobin DNA with a deletion causing India...
Sequence length 2337, 1 features, from: Homo sapiens (human)
M37467.1 Human hemoglobin-related sequence across the break...
Sequence length 1552, 1 features, from: Homo sapiens (human)


In [ ]:
handle = Entrez.efetch(db='nucleotide', rettype='gb', retmode='text', id='1515564438')
seq = SeqIO.read(handle, 'gb')
print(seq)

ID: MH580289.1
Name: MH580289
Description: Homo sapiens voucher ATGLAB 2018103 hemoglobin beta subunit (HBB) gene, exon 3 and partial cds
Number of features: 5
/molecule_type=DNA
/topology=linear
/data_file_division=PRI
/date=20-NOV-2018
/accessions=['MH580289']
/sequence_version=1
/keywords=['']
/source=Homo sapiens (human)
/organism=Homo sapiens
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
/references=[Reference(title='Homo sapiens hemoglobin subunit beta globin gene exon 3 region from homozygous sickle cell disease Indian patient', ...), Reference(title='Direct Submission', ...)]
/structured_comment=OrderedDict([('Assembly-Data', OrderedDict([('Sequencing Technology', 'Sanger dideoxy sequencing')]))])
Seq('GTGCTTATTTGCATATTCATAATCTCCCTACTTTATTTTCTTTTATTTTTAATT...GCC', IUPACAmbiguousDNA())


In [ ]:
from Bio.Seq import Seq

**Земање на листа идентификациски броеви од добиените рекорди со Ентрез пребарувањето.**

In [ ]:
record_IDs = [id for id in record['IdList']]
print(record_IDs)

['1515564438', '179408', '224959855', '2168937', '183859', '183844']


**Ги извлекуваме секвенциите од efetch пребарувањето и ги ставаме во листата se_ID**

In [ ]:
se_ID=[]
for id in record_IDs:
    handle = Entrez.efetch(db='nucleotide', rettype='gb', retmode='text', id=id)
    se_ID.append(SeqIO.read(handle,'gb').seq)
print(se_ID)    

[Seq('GTGCTTATTTGCATATTCATAATCTCCCTACTTTATTTTCTTTTATTTTTAATT...GCC', IUPACAmbiguousDNA()), Seq('ATGGTNCAYYTNACNCCNGTGGAGAAGTCYGCYGTNACNGCNCTNTGGGGYAAG...TTT', IUPACAmbiguousDNA()), Seq('CGGCTGACAAAAGAAGTCCTGGTATCTTCTATGGTGGGAGAGGAAAACTAGCTA...TGA', IUPACAmbiguousDNA()), Seq('AACCTCAAACAGACACCATGGTGCACCTGACTCCTGAGGAGAAGTCTGCC', IUPACAmbiguousDNA()), Seq('AAGCTTGGGTTTTGAGTTTTCATTATTTACCCAAAATTCATTCAGGAGCAGGTT...AAA', IUPACAmbiguousDNA()), Seq('TTTTCTCAGTCAGTTAACATTCCTTCAACTAGATTAGTTGTGACAAAATTTCAG...TGA', IUPACAmbiguousDNA())]


In [ ]:
exons = []
index = 0
with Entrez.efetch(db='nucleotide', rettype='gb', retmode='text', id=record_IDs) as handle:
  for rec in SeqIO.parse(handle, 'gb'):
    #print(rec.features)
    #print(rec.features.pop())
    for item in rec.features:
      #print(item)
      if item.type == 'exon':
        exons.append((item,index))
    index+=1

print(exons[0][0].location, exons[0][1])
# ======================================================================================= #

[499:>628](+) 0


**Извлечените секвенци ги трансформираме во SeqRecord објекти за да можеме да ги запишеме во фаста форматот.**

In [ ]:
from Bio.SeqRecord import SeqRecord

new_seq_records = []
for seq, id in zip(se_ID, record_IDs):
  new_seq_records.append(SeqRecord(seq, id))

with open('output.fa', 'w') as out:
  SeqIO.write(new_seq_records, out, 'fasta')

In [ ]:
def locs(string):
  start = ''
  end = ''
  flag = 0
  for item in string:
    if item == ':':
      flag = 1
    if flag:
      if item.isdigit():
        end+=(item)
    else:
      if item.isdigit():
        start+=(item)
  return start, end

string = '[12:>23](+)'
print(locs(string))

('12', '23')


In [ ]:
finished = []
sequen = []

for item in exons:
  print("Record with Exon: ",new_seq_records[item[1]], sep='\n')
  print("Location: ",item[0].location)
  string = str(item[0].location)
  start, end = locs(string)
  #print(start, end)
  print("Sequence: ", new_seq_records[item[1]].seq)
  print("Exon: ", new_seq_records[item[1]].seq[int(start):int(end)])
  print("Transcribed: ", new_seq_records[item[1]].seq[int(start):int(end)].transcribe())
  print("Translated: ", new_seq_records[item[1]].seq[int(start):int(end)].transcribe().translate())
  finished.append(new_seq_records[item[1]].seq[int(start):int(end)].transcribe().translate())
  sequen.append(new_seq_records[item[1]].seq[int(start):int(end)])

Record with Exon: 
ID: 1515564438
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('GTGCTTATTTGCATATTCATAATCTCCCTACTTTATTTTCTTTTATTTTTAATT...GCC', IUPACAmbiguousDNA())
Location:  [499:>628](+)
Sequence:  GTGCTTATTTGCATATTCATAATCTCCCTACTTTATTTTCTTTTATTTTTAATTGATACATAATCATTATACATATTTATGGGTTAAAGTGTAATGTTTTAATATGTGTACACATATTGACCAAATCAGGGTAATTTTGCATTTGTAATTTTAAAAAATGCTTTCTTCTTTTAATATACTTTTTTGTTTATCTTATTTCTAATACTTTCCCTAATCTCTTTCTTTCAGGGCAATAATGATACAATGTATCATGCCTCTTTGCACCATTCTAAAGAATAACAGTGATAATTTCTGGGTTAAGGCAATAGCAATATTTCTGCATATAAATATTTCTGCATATAAATTGTAACTGATGTAAGAGGTTTCATATTGCTAATAGCAGCTACAATCCAGCTACCATTCTGCTTTTATTTTATGGTTGGGATAAGGCTGGATTATTCTGAGTCCAAGCTAGGCCCTTTTGCTAATCATGTTCATACCTCTTATCTTCCTCCCACAGCTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGGGGATATTATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGCAAT

**EXON OF SICKLE CELLED ANEMIA**

In [ ]:
finished

[Seq('LLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH*', HasStopCodon(ExtendedIUPACProtein(), '*'))]

In [ ]:
sequen

[Seq('CTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAGAATTC...TAA', IUPACAmbiguousDNA())]

In [ ]:
  with open('output.fa','w') as out:
    for id in record_IDs:
      handle = Entrez.efetch(db='nucleotide', rettype='fasta', retmode='text', id=id)
      SeqIO.write(SeqIO.read(handle,'fasta'), out, 'fasta')

**Проверка дали исправно е запишано**

In [ ]:
with open('output.fa', 'r') as out:
  for handle in SeqIO.parse(out, 'fasta'):
    print(handle.seq.transcribe())

GUGCUUAUUUGCAUAUUCAUAAUCUCCCUACUUUAUUUUCUUUUAUUUUUAAUUGAUACAUAAUCAUUAUACAUAUUUAUGGGUUAAAGUGUAAUGUUUUAAUAUGUGUACACAUAUUGACCAAAUCAGGGUAAUUUUGCAUUUGUAAUUUUAAAAAAUGCUUUCUUCUUUUAAUAUACUUUUUUGUUUAUCUUAUUUCUAAUACUUUCCCUAAUCUCUUUCUUUCAGGGCAAUAAUGAUACAAUGUAUCAUGCCUCUUUGCACCAUUCUAAAGAAUAACAGUGAUAAUUUCUGGGUUAAGGCAAUAGCAAUAUUUCUGCAUAUAAAUAUUUCUGCAUAUAAAUUGUAACUGAUGUAAGAGGUUUCAUAUUGCUAAUAGCAGCUACAAUCCAGCUACCAUUCUGCUUUUAUUUUAUGGUUGGGAUAAGGCUGGAUUAUUCUGAGUCCAAGCUAGGCCCUUUUGCUAAUCAUGUUCAUACCUCUUAUCUUCCUCCCACAGCUCCUGGGCAACGUGCUGGUCUGUGUGCUGGCCCAUCACUUUGGCAAAGAAUUCACCCCACCAGUGCAGGCUGCCUAUCAGAAAGUGGUGGCUGGUGUGGCUAAUGCCCUGGCCCACAAGUAUCACUAAGCUCGCUUUCUUGCUGUCCAAUUUCUAUUAAAGGUUCCUUUGUUCCCUAAGUCCAACUACUAAACUGGGGGAUAUUAUGAAGGGCCUUGAGCAUCUGGAUUCUGCCUAAUAAAAAACAUUUAUUUUCAUUGCAAUGAUGUAUUUAAAUUAUUUCUGAAUAUUUUACUAAAAAGGGAAUGUGGGAGGUCAGUGCAUUUAAAACAUAAAGAAAUGAAGAGCUAGUUCAAACCUUGGGAAAUAACCUAAGCAUGCC
AUGGUNCAYYUNACNCCNGUGGAGAAGUCYGCYGUNACNGCNCUNUGGGGYAAGGUNAAYGUGGAUGAAGYYGGYGGYGAGGCCCUGGGCAGNCUGCUNGUGGUCUACCCUUGGACCC

In [ ]:
with open('/content/NM_000518.5.fa','r') as handle:
   for read in SeqIO.parse(handle, 'fasta'):
     sequence = read.seq
handle.close()


with open('/content/NM_000518.5.exons.fa', 'r') as handle:
  for read in SeqIO.parse(handle, 'fasta'):
    print(read.seq.transcribe().translate())
    sequence2 = read.seq.transcribe().translate()
handle.close()

TFASDTTVFTSNLKQTPWCI*LLRRSLPLLPCGAR*TWMKLVVRPWA
AAGGLPLDPEVL*VLWGSVHS*CCYGQP*GEGSWQESARCL**WPGSPGQPQGHLCHTE*AAL*QAARGS*ELQ
LLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH*ARFLAVQFLLKVPLFPKSNY*TGGYYEGP*ASGFCLIKNIYFHC


/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2742: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [ ]:
healthy = 'LLGNVLVCVLAHHFGKQFTPPVQAAYQKVVAGVANALAHKYH*' 
print(healthy,str(finished[0]),sep='\n')

LLGNVLVCVLAHHFGKQFTPPVQAAYQKVVAGVANALAHKYH*
LLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH*


In [ ]:
from Bio import AlignIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment 
healthy_seq='''CTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAACAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGGGGATATTATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGC'''
alignments = pairwise2.align.globalxx(healthy_seq,str(sequen[0]))
for alignment in alignments: 
  print(format_alignment(*alignment))
    

CTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAC-AATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGGGGATATTATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGC
||||||||||||||||||||||||||||||||||||||||||||||||  |||||||||||||||||||||||||||||||||||||||||||     |||    ||   | |       |     ||||              ||        |           ||  ||        |  |  |  |   ||               |        ||    |  |   |  |         ||   ||     |    
CTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAA-GAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGC-----TGG----TG---T-G-------G-----CTAA--------------TG--------C-----------CC--TG--------G--C--C--C---AC---------------A--------AG----T--A---T--C---------AC---TA-----A----
  Score=128

CTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAACAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGG

**=========================PATTERN MATCHING====================================================**

In [ ]:
sequence = str(sequence)
sequence[sequence.rfind('ATG'):]

'ATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGCAA'